# Fourier transform

In [ ]:
using Revise
using Plots
using LinearAlgebra
using SparseIR
import SparseIR: valueim

newaxis = [CartesianIndex()]

In [ ]:
BLAS.set_num_threads(16)

In [ ]:
println(Threads.nthreads())

In [ ]:
using ITensors

println(ITensors.blas_get_num_threads())

We want create a MPO for Fourier transform:
$$
F(t) = \sum_{x=0}^{N-1} f(x) e^{-i \frac{2\pi t x}{N}} = \sum_{x=0}^{N-1} T(t, x) f(x).
$$

MPS/MPO tensors are indexed from the left to the right in ascending order.
We assign the least significant digit to the first or the last tensor.
Let us stick to the former convention.

$x=0,...., 2^Q-1$ can be represented as a binary number, $0b001, 0b010, ..., 0b111 (=0b x_{Q-1} x_{Q-2} ... x_0)$ for $Q=3$.

$$
F(t_0, \cdots, t_{Q-1}) = \sum_{x_0=0}^1 \cdots \sum_{x_{Q-1}=0}^1  T(t_0, \cdots, T_{Q-1}, x_0, \cdots, x_{Q-1}) f(x_0, \cdots, x_{Q-1}).
$$

We transpose the tensor $T$ as $\bar T$:

$$
\bar T(t_0, x_0, \cdots, T_{Q-1}, x_{Q-1}) = T(t_0, \cdots, T_{Q-1}, x_0, \cdots, x_{Q-1})
$$

In [ ]:
nbit = 8
N = 2^nbit
tmat = zeros(ComplexF64, N, N)

for t in 0:N-1, x in 0:N-1
    tmat[t+1, x+1] = exp(-im * 2π * t * x/N)
end
#@show tmat

tmat = reshape(tmat, ntuple(x->2, 2*nbit))
# At point, the indices of `tmat` is (x_0, ..., x_{Q-1}, t_0, ..., t_{Q-1})
;

In [ ]:
# We permutate dimensions to have (x_{Q-1}, t_0, ..., x_0, t_{Q-1})
#
# Note:
# If we align the indices as (x_0, t_0, ..., x_{Q-1}, t_{Q-1}), there is no low-rank structure.
# x_0 and t_{Q-1} must be paired.
dims = Int[]
for i in 1:nbit
    push!(dims, nbit+1-i)
    push!(dims, i+nbit)
end

tmat2 = permutedims(tmat, dims)
tmat2 = reshape(tmat2, N, N)
;

In [ ]:
svd_res = svd(tmat2)
svd_res.S